# Наследование в Python

### Облагораживаем код

Пусть у нас есть два фрагмента кода, которые выкачивают Ленту.ру и N+1. Как это выглядит показано ниже. Давайте попытаемся сделать что-то получше с учетом того, что на свете существует наследование.

In [1]:
import re # Регулярные выражения.
import requests # Загрузка новостей с сайта.
from bs4 import BeautifulSoup # Превращалка html в текст.
import pymorphy2 # Морфологический анализатор.
import datetime # Новости будем перебирать по дате.
from collections import Counter # Не считать же частоты самим.
import math # Корень квадратный.

Это класс для загрузки Ленты.ру. Он позволяет загрузить статьи с сайта, сохранить их в файл считать этот файл, создать частотный словарь новостей.

Класс хранит все новости, заголовки и словари в отдельных списках.

Сайт удобен тем, что по определенному адресу (https://lenta.ru/news/год/месяц/день/) доступны ссылки на все статьи за данный день. Они хранят архив с 1999 года.

In [2]:
class getNewsPaper:
        
    # Конструктор - вызывается при создании объекта и инициализирует его.
    def __init__(self, filename=None):
        self.articles=[]     # Загруженные статьи.
        self.titles=[]       # Заголовки статей.
        self.dictionaries=[] # Словари для каждой из статей.
        # Создаем и загружаем морфологический словарь.
        self.morph=pymorphy2.MorphAnalyzer()
        if filename!=None:
            self.loadArticles(filename)

    # Загрузка статьи по URL.
    def getLentaArticle(self, url):
        """ getLentaArticle gets the body of an article from Lenta.ru"""
        # Получает текст страницы.
        resp=requests.get(url)
        # Загружаем текст в объект типа BeautifulSoup.
        bs=BeautifulSoup(resp.text, "html5lib") 
        # Получаем заголовок статьи.
        self.titles.append(bs.h1.text.replace("\xa0", " "))
        # Получаем текст статьи.
        self.articles.append(BeautifulSoup(" ".join(
                    [p.text for p in bs.find_all("p")]), "html5lib").get_text().replace("\xa0", " "))

    # Загрузка всех статей за один день.
    def getLentaDay(self, url):
        """ Gets all URLs for a given day and gets all texts. """
        try:
            # Грузим страницу со списком всех статей.
            day = requests.get(url) 
            # Получаем фрагменты с нужными нам адресами статей.
            h3s=BeautifulSoup(day.text, "html5lib").find_all("h3")
            # Получаем все адреса на статьи за день.
            links=["http://lenta.ru"+l.find_all("a")[0]["href"] for l in h3s]
            # Загружаем статьи.
            for l in links:
                self.getLentaArticle(l)
        except:
            pass

    # Загрузка всех статей за несколько дней.
    def getLentaPeriod(self, start, finish):
        curdate=start
        while curdate<=finish:
            print(curdate.strftime('%Y/%m/%d')) # Just in case.
            # Список статей грузится с вот такого адреса.
            self.getLentaDay('https://lenta.ru/news/'+curdate.strftime('%Y/%m/%d'))
            curdate+=datetime.timedelta(days=1)

    # Построение вектора для статьи.
    posConv={'ADJF':'_ADJ','NOUN':'_NOUN','VERB':'_VERB'}
    def getArticleDictionary(self, text, needPos=None):
        words=[a[0] for a in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text)]
        reswords=[]
    
        for w in words:
            wordform=self.morph.parse(w)[0]
            try:
                if wordform.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                    if needPos!=None:
                        reswords.append(wordform.normal_form+self.posConv[wordform.tag.POS])
                    else:
                        reswords.append(wordform.normal_form)
            except:
                pass
            
        stat=Counter(reswords)
        # Берем только слова с частотой больше 1.
        stat={a: stat[a] for a in stat.keys() if stat[a]>1}
        return stat

    # Посчитаем вектора для всех статей.
    def calcArticleDictionaries(self, needPos=None):
        self.dictionaries=[]
        for a in self.articles:
            self.dictionaries.append(self.getArticleDictionary(a, needPos))
            
    # Сохраняем статьи в файл.
    def saveArticles(self, filename):
        """ Saves all articles to a file with a filename. """
        newsfile=open(filename, "w")
        for art, titl in zip(self.articles, self.titles):
            newsfile.write('\n=====\n'+titl)
            newsfile.write('\n-----\n'+art)
        newsfile.close()

    # Читаем статьи из файла.
    def loadArticles(self, filename):
        """ Loads and replaces all articles from a file with a filename. """
        newsfile=open(filename, encoding="utf-8")
        text=newsfile.read()
        self.articles=text.split('\n=====\n')[1:]
        for i, a in enumerate(self.articles):
            b, self.articles[i] = a.split('\n-----\n')
            self.titles.append(b)
        newsfile.close()

    # Для удобства - поиск статьи по ее заголовку.
    def findNewsByTitle(self, title):
        if title in self.titles:
            return self.titles.index(title)
        else:
            return -1

Это класс для загрузки сайта N+1.ru. Его структура очень сильно схожа со структурой Ленты.ру, именно поэтому мы его и берем. 

Здесь заведен тип под новостную заметку, так как для нее хранится гораздо больше информации: время, дата, рубрика, сложность, автор, заголовок, текст. Помимо этого, статья умеет сохранять себя в [JSON](https://ruseller.com/lessons.php?id=1212) (без использования соответствующей [библиотеки](https://pythonworld.ru/moduli/modul-json.html)) и словарь Питона.

Дальше идут две функции, которые выгружают отдельную статью и все статьи за день.

In [3]:
delcom=re.compile("<!--.+-->", re.S)

# Класс, хранящий информацию о статье.
class NPlus1Article:
    def __init__(self):
        self.time=""
        self.date=""
        self.rubr=""
        self.diff=""
        self.author=""
        self.title=""
        self.text=""
        
    # Конвертация в JSON.
    def toJSON(self):
        res='{"date":"'+self.date+'", "time":"'+self.time+'", "rubrics":"'+self.rubr+'", "difficulty":"'
        res+=self.diff+'", "title":"'+self.head+'", "author":"'+self.author+'","text":"'
        res+=self.text.replace('"', '\\"')+'"}'
        return res

    # Конвертация в словарь.
    def toDict(self):
        res={"date":self.date, "time":self.time, "rubrics":self.rubr, "difficulty":self.diff,\
             "title":self.head, "author":self.author,"text":self.text.replace('"', '\\"')}
        return res
    
def getArticleTextNPlus1(adr):
    r = requests.get(adr)
    #print(r.text)
    art = NPlus1Article()
    tables = re.split("</div>",re.split('="tables"', r.text)[1])[0]
    t1 = re.split("</time>", re.split("<time", tables)[1])[0]
    art.time = re.split("</span>", re.split("<span>", t1)[1])[0]
    art.date = re.split("</span>", re.split("<span>", t1)[2])[0]
    art.rubr = re.findall("<a data-rubric.+?>(.+?)</a>", r.text)[0]
    art.diff = re.split("</span>", re.split('"difficult-value">', tables)[1])[0]
    art.head = re.split("</h1>",re.split('<h1>', r.text)[1])[0]
    art.author = re.split('" />',re.split('<meta name="author" content="', r.text)[1])[0]
    art.text = re.split("</div>", re.split("</figure>", re.split('</article>',re.split('<article', r.text)[1])[0])[1])[1]    

    beaux_text = BeautifulSoup(art.text, "html5lib")
    art.text = delcom.sub("", beaux_text.get_text() )
    art.text = art.text.replace('\xa0', ' ')

    # print(art.n_time, art.n_date, art.n_rubr, art.n_diff)
    # print(art.n_head)
    # print(art.n_author)
    # print(art.n_text)
    #return [n_time, n_date, n_rubr, n_diff, n_author, n_head, n_text]
    return art

def getDayArticles(adr):
    r = requests.get(adr)
    titles = BeautifulSoup(r.text, "html5lib")("article")
    #print(titles)
    addrs = ["https://nplus1.ru/"+a("a")[0]["href"] for a in titles]
    #print(addrs)
    articles = []
    for adr in addrs:
        articles.append(getArticleTextNPlus1(adr))
    return articles

Давайте для начала откажемся от классов, которые выкачивают данные, и сосредоточимся на самих данных, которые они возвращают. Давайте считать, что заголовок и текст есть у любой статьи, а всё остальное - это дополнительные детали, зависящие от конкретного сайта.

Создадим класс базовой статьи, который будет содержать в себе только два поля. [Унаследуем](https://o7planning.org/ru/11417/inheritance-and-polymorphism-in-python) от него класс статьи для N+1 - NPlus1Article. Теперь можно считать, что любая функция возвращает объект, который ведет себя как объект BaseArticle - у него есть поля title и text.

В конструкторе NPlus1Article вызывается конструктор базового класса. При помощи функции `super()` мы обращаемся к объекты как к объекту родительского класса (нам даже не важно знать какого).

----

Вообще, наследование необходимо для трех вещей:
- расширить функционал или набор данных имеющегося класса;
- взять несколько классов с общей частью и выделить ее в единый базовый класс **с выделением соответствующей сущности**.
- обеспечить единый интерфейс для классов-наследников (это скорее подход для [языков группы Вирта](https://habr.com/ru/post/303380/), в Питоне часто используется интерфейс по договоренности, но наследование помогает упросить понимание).

In [4]:
class BaseArticle:
    def __init__(self):
        self.title=""
        self.text=""
        
class NPlus1Article(BaseArticle):
    def __init__(self):
        super().__init__()
        self.time=""
        self.date=""
        self.rubr=""
        self.diff=""
        self.author=""
        
    def getSuper(self):
        return super(NPlus1Article, self)
        


In [5]:
rr=NPlus1Article()
rr.getSuper()

<super: __main__.NPlus1Article, <__main__.NPlus1Article at 0x7f940407df60>>

Оформим загрузку статей как функции и напишем еще одну, которая выгружает статью в JSON, список выгружаемых полей зависит от типа переменной. Проверка типа проводится при помощи функции `isinstance`.

In [10]:
def getArticleTextNPlus1(adr):
    r = requests.get(adr)
    #print(r.text)
    art = NPlus1Article()
    tables = re.split("</div>",re.split('="tables"', r.text)[1])[0]
    t1 = re.split("</time>", re.split("<time", tables)[1])[0]
    art.time = re.split("</span>", re.split("<span>", t1)[1])[0]
    art.date = re.split("</span>", re.split("<span>", t1)[2])[0]
    art.rubr = re.findall("<a data-rubric.+?>(.+?)</a>", r.text)[0]
    art.diff = re.split("</span>", re.split('"difficult-value">', tables)[1])[0]
    art.head = re.split("</h1>",re.split('<h1>', r.text)[1])[0]
    art.author = re.split('" />',re.split('<meta name="author" content="', r.text)[1])[0]
    art.text = re.split("</div>", re.split("</figure>", re.split('</article>',re.split('<article', r.text)[1])[0])[1])[1]    

    beaux_text = BeautifulSoup(art.text, "html5lib")
    art.text = delcom.sub("", beaux_text.get_text() )
    art.text = art.text.replace('\xa0', ' ')

    # print(art.n_time, art.n_date, art.n_rubr, art.n_diff)
    # print(art.n_head)
    # print(art.n_author)
    # print(art.n_text)
    #return [n_time, n_date, n_rubr, n_diff, n_author, n_head, n_text]
    return art

def getLentaArticle(url):
    """ getLentaArticle gets the body of an article from Lenta.ru"""
    # Получает текст страницы.
    resp=requests.get(url)
    # Загружаем текст в объект типа BeautifulSoup.
    bs=BeautifulSoup(resp.text, "html5lib")
    art=BaseArticle()
    art.title=bs.h1.text.replace("\xa0", " ")
    art.text=BeautifulSoup(" ".join([p.text for p in bs.find_all("p")]), "html5lib").get_text().replace("\xa0", " ")
    return art

def articleToJSON(art):
    # Здесь сделаем неправильно - посмотрим на isinstance(BaseArticle), а потом elif(NPlus1Article), тогда в else не пойдет.
    if isinstance(art , BaseArticle):
        return '{"title":"'+art.title+art.text.replace('"', '\\"')+'"}'
    elif isinstance(art , NPlus1Article):
        res='{"date":"'+art.date+'", "time":"'+art.time+'", "rubrics":"'+art.rubr+'", "difficulty":"'
        res+=art.diff+'", "title":"'+art.title+'", "author":"'+art.author+'","text":"'
        res+=art.text.replace('"', '\\"')+'"}'
        return res


Загружаем статьи и выводим JSON.

In [11]:
a1=getLentaArticle("https://lenta.ru/news/2019/02/20/trash/")
a2=getArticleTextNPlus1("https://nplus1.ru/news/2019/02/20/deep-sqeak")

print(articleToJSON(a1))
print(articleToJSON(a2))

{"title":"Германия научит Россию собирать мусорБывший министр Германии по вопросам окружающей среды, охраны природы и защиты атомных реакторов Клаус Тепфер проконсультирует российское Минприроды по поводу утилизации мусора, пишет РБК со ссылкой на пресс-службу вице-премьера Алексея Гордеева. «Главный идеолог немецкой модели переработки и утилизации мусора выступит независимым консультантом запуска российской системы обращения с твердыми коммунальными отходами и будет экспертно сопровождать ход ее реализации», — говорится в сообщении по итогам визита Гордеева в Германию. Отмечается, что во время поездки вице-премьер ознакомился с немецкой системой сбора и переработки мусора. Благодаря ей все отходы удается перерабатывать во вторсырье, «зеленый уголь» и цветочный грунт. В России этот путь проделывают только 10 процентов отходов. По словам Гордеева, российские власти ставят перед собой задачу в несколько раз повысить эффективность переработки отходов за ближайшие шесть лет, а также полнос

Оказывается, функция `isinstance` проверяет не приводится ли объект к проверяемому типу и если приводится, то возвращает True. Нам же необходимо проверить точное совпадение с типом. Для этого будем использовать конструкцию `type(a) is`. 

Можно, конечно, просто расставить проверки в правильном порядке, оставив базовый класс напоследок, но так больше шанс наделать ошибок.

In [12]:
def articleToJSON(art):
    # Здесь сделаем неправильно - посмотрим на isinstance(BaseArticle), а потом elif(NPlus1Article), тогда в else не пойдет.
    if type(art) is BaseArticle:
        return '{"title":"'+art.title+art.text.replace('"', '\\"')+'"}'
    elif type(art) is NPlus1Article:
        res='{"date":"'+art.date+'", "time":"'+art.time+'", "rubrics":"'+art.rubr+'", "difficulty":"'
        res+=art.diff+'", "title":"'+art.title+'", "author":"'+art.author+'","text":"'
        res+=art.text.replace('"', '\\"')+'"}'
        return res

a1=getLentaArticle("https://lenta.ru/news/2019/02/20/trash/")
a2=getArticleTextNPlus1("https://nplus1.ru/news/2019/02/20/deep-sqeak")

print(articleToJSON(a1))
print(articleToJSON(a2))

{"title":"Германия научит Россию собирать мусорБывший министр Германии по вопросам окружающей среды, охраны природы и защиты атомных реакторов Клаус Тепфер проконсультирует российское Минприроды по поводу утилизации мусора, пишет РБК со ссылкой на пресс-службу вице-премьера Алексея Гордеева. «Главный идеолог немецкой модели переработки и утилизации мусора выступит независимым консультантом запуска российской системы обращения с твердыми коммунальными отходами и будет экспертно сопровождать ход ее реализации», — говорится в сообщении по итогам визита Гордеева в Германию. Отмечается, что во время поездки вице-премьер ознакомился с немецкой системой сбора и переработки мусора. Благодаря ей все отходы удается перерабатывать во вторсырье, «зеленый уголь» и цветочный грунт. В России этот путь проделывают только 10 процентов отходов. По словам Гордеева, российские власти ставят перед собой задачу в несколько раз повысить эффективность переработки отходов за ближайшие шесть лет, а также полнос

Так работает корректней.

Мы увидели, что наследование может использоваться для расширения функционала. Попробуем применить это знание к проектированию классов, которые загружают статьи. Создадим базовый класс, который умеет сохранять статьи в файл и считывать их оттуда. При этом сохраняться будут только заголовки и текст, а остальная информация, если она была, будет теряться. Также класс будет уметь строить частотные словари для статей. Дальше унаследуемся от этого класса и добавим функции работы с заданным сайтом.

In [15]:
class BaseGetNewsPaper:
    ''' Базовый класс для загрузки статей. Обеспечивает основной функционал, но не интерфейс.
    '''        
    # Конструктор - вызывается при создании объекта и инициализирует его.
    def __init__(self):
        self.articles=[]     # Загруженные статьи.
        self.dictionaries=[] # Словари для каждой из статей.
        # Создаем и загружаем морфологический словарь.
        self.morph=pymorphy2.MorphAnalyzer()

    # Построение вектора для статьи.
    def getArticleDictionary(self, text, needPos=None):
        words=[a[0] for a in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text)]
        reswords=[]
    
        for w in words:
            wordform=self.morph.parse(w)[0]
            try:
                if wordform.tag.POS in ['ADJF', 'NOUN', 'VERB', 'PRTF', 'GRND']:
                    if needPos!=None:
                        reswords.append(wordform.normal_form+'_'+wordform.tag.POS)
                    else:
                        reswords.append(wordform.normal_form)
            except:
                pass
            
        stat=Counter(reswords)
        # Берем только слова с частотой больше 1.
        stat={a: stat[a] for a in stat.keys() if stat[a]>1}
        return stat

    # Посчитаем вектора для всех статей.
    def calcArticleDictionaries(self, needPos=None):
        self.dictionaries=[]
        for a in self.articles:
            self.dictionaries.append(self.getArticleDictionary(a.text, needPos))
            
    # Сохраняем статьи в файл.
    def saveArticles(self, filename):
        """ Saves all articles to a file with a filename. """
        newsfile = open(filename, "w")
        for art in self.articles:
            newsfile.write('\n=====\n'+art.title)
            newsfile.write('\n-----\n'+art.text)
        newsfile.close()

    # Читаем статьи из файла.
    def loadArticles(self, filename):
        """ Loads and replaces all articles from a file with a filename. """
        newsfile = open(filename, encoding="utf-8")
        text = newsfile.read()
        loaded = text.split('\n=====\n')[1:]
        self.articles=[]
        for i, a in enumerate(loaded):
            self.articles.append(BaseArticle())
            b, self.articles[i].text = a.split('\n-----\n')
            self.articles[i].title = b
        newsfile.close()

class GetLenta(BaseGetNewsPaper):
    ''' Класс для загрузки Ленты.ру.  Наследуется от BaseGetNewsPaper.
    '''
    # Загрузка статьи по URL.
    def getLentaArticle(self, url):
        """ getLentaArticle gets the body of an article from Lenta.ru"""
        # Получает текст страницы.
        resp=requests.get(url)
        # Загружаем текст в объект типа BeautifulSoup.
        bs=BeautifulSoup(resp.text, "html5lib") 
        
        art=BaseArticle()
        # Получаем заголовок статьи.
        art.title=bs.h1.text.replace("\xa0", " ")
        # Получаем текст статьи.
        art.text=BeautifulSoup(" ".join([p.text for p in bs.find_all("p")]), "html5lib").get_text().replace("\xa0", " ")
        return art

    # Загрузка всех статей за один день.
    def getLentaDay(self, url):
        """ Gets all URLs for a given day and gets all texts. """
        try:
            # Грузим страницу со списком всех статей.
            day = requests.get(url) 
            # Получаем фрагменты с нужными нам адресами статей.
            h3s=BeautifulSoup(day.text, "html5lib").find_all("h3")
            # Получаем все адреса на статьи за день.
            links=["http://lenta.ru"+l.find_all("a")[0]["href"] for l in h3s]
            # Загружаем статьи.
            for l in links:
                art=self.getLentaArticle(l)
                self.articles.append(art)
        except:
            pass

    # Загрузка всех статей за несколько дней.
    def getLentaPeriod(self, start, finish):
        """Gets articles for a period fom start to finish. """
        curdate=start
        while curdate<=finish:
            print(curdate.strftime('%Y/%m/%d')) # Just in case.
            # Список статей грузится с вот такого адреса.
            self.getLentaDay('https://lenta.ru/news/'+curdate.strftime('%Y/%m/%d'))
            curdate+=datetime.timedelta(days=1)

class GetNPlus1(BaseGetNewsPaper):
    ''' Класс для загрузки NPlus1.ru. Наследуется от BaseGetNewsPaper.
    '''
    def getArticleTextNPlus1(self, adr):
        """Get an article from nplus1.ru"""
        r = requests.get(adr)
        #print(r.text)
        art = NPlus1Article()
        tables = re.split("</div>",re.split('="tables"', r.text)[1])[0]
        t1 = re.split("</time>", re.split("<time", tables)[1])[0]
        art.time = re.split("</span>", re.split("<span>", t1)[1])[0]
        art.date = re.split("</span>", re.split("<span>", t1)[2])[0]
        art.rubr = re.findall("<a data-rubric.+?>(.+?)</a>", r.text)[0]
        art.diff = re.split("</span>", re.split('"difficult-value">', tables)[1])[0]
        art.title = re.findall("<h1>(.+?)</h1>", r.text)[0]
        art.author = re.split('" />',re.split('<meta name="author" content="', r.text)[1])[0]
        art.text = re.split("</div>", re.split("</figure>", re.split('</article>',re.split('<article', r.text)[1])[0])[1])[1]    

        beaux_text = BeautifulSoup(art.text, "html5lib")
        art.text = delcom.sub("", beaux_text.get_text() )
        art.text = art.text.replace('\xa0', ' ')
        return art

    def getNPlus1Day(self, adr):
        """ Gwt all article for a day by its URL given in adr parameter."""
        r = requests.get(adr)
        titles = BeautifulSoup(r.text, "html5lib")("article")
        addrs = ["https://nplus1.ru/"+a("a")[0]["href"] for a in titles]
        for adr in addrs:
            aa=self.getArticleTextNPlus1(adr)
            self.articles.append(aa)
        
    # Загрузка всех статей за несколько дней.
    def getNPlus1Period(self, start, finish):
        """ Gets all articles from nplus1.ru for a period from start to finish. """
        curdate=start
        while curdate<=finish:
            print(curdate.strftime('%Y/%m/%d')) # Just in case.
            # Список статей грузится с вот такого адреса.
            self.getNPlus1Day('https://nplus1.ru/news/'+curdate.strftime('%Y/%m/%d'))
            curdate+=datetime.timedelta(days=1)



Протестируем как работают наши классы - загрузим три дня с обоих сайтов.

In [16]:
lenta=GetLenta()
lenta.getLentaPeriod(datetime.date(2018, 2, 1), datetime.date(2018, 2, 3))
lenta.saveArticles("data/lenta_test.txt")

2018/02/01
2018/02/02
2018/02/03


In [17]:
n1=GetNPlus1()
n1.getNPlus1Period(datetime.date(2018, 2, 1), datetime.date(2018, 2, 3))

n1.saveArticles("data/nplus1_test.txt")


2018/02/01
2018/02/02
2018/02/03


Прочитаем статьи из файла.

In [18]:
n2=GetNPlus1()
n2.loadArticles("data/nplus1_test.txt")

n2.articles[1].title

'ДНК помогла управлять роем молекулярных моторов из микротрубочек'

Однако давать разные названия для функций, которые делают одно и то же - не очень хорошо, особенно если эти функции входят в интерфейс класса.

Интерфейс (в случае Python) - это некоторые обязательства, что класс умеет выполнять определенные функции. В этом случае не важно, какой именно класс наследует. Для этого необходимо у базового класса определить необходимые функции, а в дочерних классах переопределить эти функции.

In [19]:
class BaseGetNewsPaper:
    ''' Базовый класс для загрузки статей. Обеспечивает основной функционал и интерфейс.
        Классы-наследники будут уметь загружать данные при помощи одного программного интерфейса.
    '''        
        
    # Конструктор - вызывается при создании объекта и инициализирует его.
    def __init__(self):
        self.articles=[]     # Загруженные статьи.
        self.dictionaries=[] # Словари для каждой из статей.
        # Создаем и загружаем морфологический словарь.
        self.morph=pymorphy2.MorphAnalyzer()

    # Построение вектора для статьи.
    def getArticleDictionary(self, text, needPos=None):
        words=[a[0] for a in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text)]
        reswords=[]
    
        for w in words:
            wordform=self.morph.parse(w)[0]
            try:
                if wordform.tag.POS in ['ADJF', 'NOUN', 'VERB', 'PRTF', 'GRND']:
                    if needPos!=None:
                        reswords.append(wordform.normal_form+'_'+wordform.tag.POS)
                    else:
                        reswords.append(wordform.normal_form)
            except:
                pass
            
        stat=Counter(reswords)
        # Берем только слова с частотой больше 1.
        stat={a: stat[a] for a in stat.keys() if stat[a]>1}
        return stat

    # Посчитаем вектора для всех статей.
    def calcArticleDictionaries(self, needPos=None):
        self.dictionaries=[]
        for a in self.articles:
            self.dictionaries.append(self.getArticleDictionary(a.text, needPos))
            

    def getPeriod(self, start, finish):
        print("Nothing to do.")
    
    # Сохраняем статьи в файл.
    def saveArticles(self, filename):
        """ Saves all articles to a file with a filename. """
        newsfile = open(filename, "w")
        for art in self.articles:
            newsfile.write('\n=====\n'+art.title)
            newsfile.write('\n-----\n'+art.text)
        newsfile.close()

    # Читаем статьи из файла.
    def loadArticles(self, filename):
        """ Loads and replaces all articles from a file with a filename. """
        newsfile = open(filename, encoding="utf-8")
        text = newsfile.read()
        loaded = text.split('\n=====\n')[1:]
        self.articles=[]
        for i, a in enumerate(loaded):
            self.articles.append(BaseArticle())
            b, self.articles[i].text = a.split('\n-----\n')
            self.articles[i].title = b
        newsfile.close()
        
class GetLenta(BaseGetNewsPaper):
    ''' Класс для загрузки Ленты.ру.  Наследуется от BaseGetNewsPaper.
    '''

    # Загрузка статьи по URL.
    def getLentaArticle(self, url):
        """ getLentaArticle gets the body of an article from Lenta.ru"""
        # Получает текст страницы.
        resp=requests.get(url)
        # Загружаем текст в объект типа BeautifulSoup.
        bs=BeautifulSoup(resp.text, "html5lib") 
        
        art=BaseArticle()
        # Получаем заголовок статьи.
        art.title=bs.h1.text.replace("\xa0", " ")
        # Получаем текст статьи.
        art.text=BeautifulSoup(" ".join([p.text for p in bs.find_all("p")]), "html5lib").get_text().replace("\xa0", " ")
        return art

    # Загрузка всех статей за один день.
    def getLentaDay(self, url):
        """ Gets all URLs for a given day and gets all texts. """
        try:
            # Грузим страницу со списком всех статей.
            day = requests.get(url) 
            # Получаем фрагменты с нужными нам адресами статей.
            h3s=BeautifulSoup(day.text, "html5lib").find_all("h3")
            # Получаем все адреса на статьи за день.
            links=["http://lenta.ru"+l.find_all("a")[0]["href"] for l in h3s]
            # Загружаем статьи.
            for l in links:
                art=self.getLentaArticle(l)
                self.articles.append(art)
        except:
            pass

    # Загрузка всех статей за несколько дней.
    def getLentaPeriod(self, start, finish):
        curdate=start
        while curdate<=finish:
            print(curdate.strftime('%Y/%m/%d')) # Just in case.
            # Список статей грузится с вот такого адреса.
            self.getLentaDay('https://lenta.ru/news/'+curdate.strftime('%Y/%m/%d'))
            curdate+=datetime.timedelta(days=1)

    def getPeriod(self, start, finish):
        self.getLentaPeriod(start, finish)
    

class GetNPlus1(BaseGetNewsPaper):
    ''' Класс для загрузки NPlus1.ru.  Наследуется от BaseGetNewsPaper.
    '''

    def getArticleTextNPlus1(self, adr):
        r = requests.get(adr)
        #print(r.text)
        art = NPlus1Article()
        tables = re.split("</div>",re.split('="tables"', r.text)[1])[0]
        t1 = re.split("</time>", re.split("<time", tables)[1])[0]
        art.time = re.split("</span>", re.split("<span>", t1)[1])[0]
        art.date = re.split("</span>", re.split("<span>", t1)[2])[0]
        art.rubr = re.findall("<a data-rubric.+?>(.+?)</a>", r.text)[0]
        art.diff = re.split("</span>", re.split('"difficult-value">', tables)[1])[0]
        art.title = re.findall("<h1>(.+?)</h1>", r.text)[0]
        art.author = re.split('" />',re.split('<meta name="author" content="', r.text)[1])[0]
        art.text = re.split("</div>", re.split("</figure>", re.split('</article>',re.split('<article', r.text)[1])[0])[1])[1]    

        beaux_text = BeautifulSoup(art.text, "html5lib")
        art.text = delcom.sub("", beaux_text.get_text() )
        art.text = art.text.replace('\xa0', ' ')
        return art

    def getNPlus1Day(self, adr):
        r = requests.get(adr)
        titles = BeautifulSoup(r.text, "html5lib")("article")
        addrs = ["https://nplus1.ru/"+a("a")[0]["href"] for a in titles]
        for adr in addrs:
            aa=self.getArticleTextNPlus1(adr)
            self.articles.append(aa)
        
    # Загрузка всех статей за несколько дней.
    def getNPlus1Period(self, start, finish):
        curdate=start
        while curdate<=finish:
            print(curdate.strftime('%Y/%m/%d')) # Just in case.
            # Список статей грузится с вот такого адреса.
            self.getNPlus1Day('https://nplus1.ru/news/'+curdate.strftime('%Y/%m/%d'))
            curdate+=datetime.timedelta(days=1)

    def getPeriod(self, start, finish):
        self.getNPlus1Period(start, finish)


In [99]:
#n1=GetNPlus1()
#filename="nplus1_test.txt"
n1=GetLenta()
filename="lenta_test.txt"

А теперь попробуйте угадать для какого класса был выполнен этот код?

In [100]:

n1.getPeriod(datetime.date(2018, 2, 1), datetime.date(2018, 2, 2))
n1.saveArticles(filename)


2018/02/01
2018/02/02


### Абстрактные методы и классы

Теперь сделаем так, чтобы объекты базового класса нельзя было создавать. Это не совсем хорошо именно для нашего случая, но для учебных целей нормально.

Подключим абстрактные классы из библиотеки ABC (Abstract Base Classes) и декоратор abstractmethod. Теперь метод `getPeriod` будет абстрактным, то есть он не реализован. Из-за этого (и наследования от ABC) класс BaseGetNewsPaper будет абстрактным, то есть его объекты нельзя создавать.

In [20]:
from abc import ABC, abstractmethod

In [21]:
class BaseGetNewsPaper(ABC):
        
    # Конструктор - вызывается при создании объекта и инициализирует его.
    def __init__(self):
        self.articles=[]     # Загруженные статьи.
        self.dictionaries=[] # Словари для каждой из статей.
        # Создаем и загружаем морфологический словарь.
        self.morph=pymorphy2.MorphAnalyzer()

    # Построение вектора для статьи.
    def getArticleDictionary(self, text, needPos=None):
        words=[a[0] for a in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text)]
        reswords=[]
    
        for w in words:
            wordform=self.morph.parse(w)[0]
            try:
                if wordform.tag.POS in ['ADJF', 'NOUN', 'VERB', 'PRTF', 'GRND']:
                    if needPos!=None:
                        reswords.append(wordform.normal_form+'_'+wordform.tag.POS)
                    else:
                        reswords.append(wordform.normal_form)
            except:
                pass
            
        stat=Counter(reswords)
        # Берем только слова с частотой больше 1.
        stat={a: stat[a] for a in stat.keys() if stat[a]>1}
        return stat

    # Посчитаем вектора для всех статей.
    def calcArticleDictionaries(self, needPos=None):
        self.dictionaries=[]
        for a in self.articles:
            self.dictionaries.append(self.getArticleDictionary(a.text, needPos))
            
    @abstractmethod
    def getPeriod(self, start, finish):
        pass
    
    # Сохраняем статьи в файл.
    def saveArticles(self, filename):
        """ Saves all articles to a file with a filename. """
        newsfile = open(filename, "w")
        for art in self.articles:
            newsfile.write('\n=====\n'+art.title)
            newsfile.write('\n-----\n'+art.text)
        newsfile.close()

    # Читаем статьи из файла.
    def loadArticles(self, filename):
        """ Loads and replaces all articles from a file with a filename. """
        newsfile = open(filename, encoding="utf-8")
        text = newsfile.read()
        loaded = text.split('\n=====\n')[1:]
        self.articles=[]
        for i, a in enumerate(loaded):
            self.articles.append(BaseArticle())
            b, self.articles[i].text = a.split('\n-----\n')
            self.articles[i].title = b
        newsfile.close()
        
class GetLenta(BaseGetNewsPaper):
    # Загрузка статьи по URL.
    def getLentaArticle(self, url):
        """ getLentaArticle gets the body of an article from Lenta.ru"""
        # Получает текст страницы.
        resp=requests.get(url)
        # Загружаем текст в объект типа BeautifulSoup.
        bs=BeautifulSoup(resp.text, "html5lib") 
        
        art=BaseArticle()
        # Получаем заголовок статьи.
        art.title=bs.h1.text.replace("\xa0", " ")
        # Получаем текст статьи.
        art.text=BeautifulSoup(" ".join([p.text for p in bs.find_all("p")]), "html5lib").get_text().replace("\xa0", " ")
        return art

    # Загрузка всех статей за один день.
    def getLentaDay(self, url):
        """ Gets all URLs for a given day and gets all texts. """
        try:
            # Грузим страницу со списком всех статей.
            day = requests.get(url) 
            # Получаем фрагменты с нужными нам адресами статей.
            h3s=BeautifulSoup(day.text, "html5lib").find_all("h3")
            # Получаем все адреса на статьи за день.
            links=["http://lenta.ru"+l.find_all("a")[0]["href"] for l in h3s]
            # Загружаем статьи.
            for l in links:
                art=self.getLentaArticle(l)
                self.articles.append(art)
        except:
            pass

    # Загрузка всех статей за несколько дней.
    def getLentaPeriod(self, start, finish):
        curdate=start
        while curdate<=finish:
            print(curdate.strftime('%Y/%m/%d')) # Just in case.
            # Список статей грузится с вот такого адреса.
            self.getLentaDay('https://lenta.ru/news/'+curdate.strftime('%Y/%m/%d'))
            curdate+=datetime.timedelta(days=1)

    def getPeriod(self, start, finish):
        self.getLentaPeriod(start, finish)
    

class GetNPlus1(BaseGetNewsPaper):
    def getArticleTextNPlus1(self, adr):
        r = requests.get(adr)
        #print(r.text)
        art = NPlus1Article()
        tables = re.split("</div>",re.split('="tables"', r.text)[1])[0]
        t1 = re.split("</time>", re.split("<time", tables)[1])[0]
        art.time = re.split("</span>", re.split("<span>", t1)[1])[0]
        art.date = re.split("</span>", re.split("<span>", t1)[2])[0]
        art.rubr = re.findall("<a data-rubric.+?>(.+?)</a>", r.text)[0]
        art.diff = re.split("</span>", re.split('"difficult-value">', tables)[1])[0]
        art.title = re.findall("<h1>(.+?)</h1>", r.text)[0]
        art.author = re.split('" />',re.split('<meta name="author" content="', r.text)[1])[0]
        art.text = re.split("</div>", re.split("</figure>", re.split('</article>',re.split('<article', r.text)[1])[0])[1])[1]    

        beaux_text = BeautifulSoup(art.text, "html5lib")
        art.text = delcom.sub("", beaux_text.get_text() )
        art.text = art.text.replace('\xa0', ' ')
        return art

    def getNPlus1Day(self, adr):
        r = requests.get(adr)
        titles = BeautifulSoup(r.text, "html5lib")("article")
        addrs = ["https://nplus1.ru/"+a("a")[0]["href"] for a in titles]
        for adr in addrs:
            aa=self.getArticleTextNPlus1(adr)
            self.articles.append(aa)
        
    # Загрузка всех статей за несколько дней.
    def getNPlus1Period(self, start, finish):
        curdate=start
        while curdate<=finish:
            print(curdate.strftime('%Y/%m/%d')) # Just in case.
            # Список статей грузится с вот такого адреса.
            self.getNPlus1Day('https://nplus1.ru/news/'+curdate.strftime('%Y/%m/%d'))
            curdate+=datetime.timedelta(days=1)

    def getPeriod(self, start, finish):
        self.getNPlus1Period(start, finish)


In [22]:
# Создать не получится - теперь это абстрактный класс.
nn=BaseGetNewsPaper()


TypeError: Can't instantiate abstract class BaseGetNewsPaper with abstract methods getPeriod

In [23]:
# А здесь всё в порядке.
n1=GetLenta()
n2=GetNPlus1()

In [ ]:
n1.getPeriod(datetime.date(2018, 2, 1), datetime.date(2018, 2, 2))

Теперь добавим этим классам набор операторов, которые сделают работу ними более удобной. Хотя некоторые операторы скорее для демонстрации возможностей применения перегруженных операторов.

In [24]:
from copy import copy

In [25]:
# Базовый класс статьи, он же класс статьи для Ленты.ру
class BaseArticle:
    def __init__(self, _title=None, _text=None):
        """Конструктор для базового класса статьи. Заводит поля title и text."""
        if isinstance(_title, str) and isinstance(_text, str):
            self.title=_title
            self.text=_text
        else:
            self.title=""
            self.text=""
        
    # Конвертация в JSON.
    def toJSON(self):
        """Возвращает представление базовой статьи в формате строки JSON."""
        res='{"title":"'+self.title.replace('"', '\\"')+'","text":"'+self.text.replace('"', '\\"')+'"}'
        return res

    # Конвертация в словарь.
    def toDict(self):
        """Возвращает представление базовой статьи в виде словаря."""
        res={"title":self.title.replace('"', '\\"'), "text":self.text.replace('"', '\\"')}
        return res
    
    # Возвращает строковое представление статьи если преобразуется к строке при помощи str(article)
    # или print(article).
    def __str__(self):
        """Возвращает строку из 200 первых символов заголовка статьи и 200 первых символов самой статьи."""
        return '<title: '+self.title[:200]+'\ntext: '+self.text[:200]+'... >'

    # Возвращает строковое представление статьи если мы просим среду отобразить статью ьез print.
    def __repr__(self):
        """Возвращает строку из 100 первых символов заголовка как краткое текстовое представление статьи."""
        return '<Base Article on "'+self.title[:100]+'">'
    
    
class NPlus1Article(BaseArticle):
    def __init__(self):
        """Конструктор для базового класса статьи. Заводит поля title, text, date, time, rubr, diff и author."""
        # Вызываем конструктор от базового класса.
        super().__init__()
        self.time=""
        self.date=""
        self.rubr=""
        self.diff=""
        self.author=""
        
    # Конвертация в JSON.
    def toJSON(self):
        """Возвращает представление базовой статьи в формате строки JSON."""
        res='{"date":"'+self.date+'", "time":"'+self.time+'", "rubrics":"'+self.rubr+'", "difficulty":"'
        res+=self.diff+'", "title":"'+self.title+'", "author":"'+self.author+'","text":"'
        res+=self.text.replace('"', '\\"')+'"}'
        return res

    # Конвертация в словарь.
    def toDict(self):
        """Возвращает представление базовой статьи в виде словаря."""
        res={"date":self.date, "time":self.time, "rubrics":self.rubr, "difficulty":self.diff,\
             "title":self.title, "author":self.author,"text":self.text.replace('"', '\\"')}
        return res

    def __str__(self):
        """Возвращает строку из метаданных о статье, 200 первых символов заголовка статьи 
           и 200 первых символов самой статьи."""
        res='<date:"'+self.date+' : '+self.time+'\nrubrics: '+self.rubr+'\ndifficulty: '+self.diff+ \
            '\nauthor: '+self.author
        res+='\ntitle: '+self.title[:100]+'\ntext: '+self.text[:100]+'>'
        return res

    def __repr__(self):
        """Возвращает строку из 100 первых символов заголовка как краткое текстовое представление статьи."""
        return '<NPlus1 Article on "'+self.title[:100]+'">'
    
    
# Базовый класс для загрузчиков новостей.
# В образовательных целях сделан как абстрактный класс, то есть класс, объекты которого нельзя создавать.
# Можно унаследоваться, переопределить абстрактные функции (отмечены декоратором @abstractmethod).
# По-хорошему, можно было бы использовать для того, чтобы прочитать новости и работать с ними.
# Умеет посчитать частотные вектора статей.
class BaseGetNewsPaper(ABC):
        
    # Конструктор - вызывается при создании объекта и инициализирует его.
    def __init__(self, data=None):
        """Конструктор объектов класса BaseGetNewsPaper. 
           Может принимать инициализирующие параметры типа BaseGetNewsPaper (создает копию)
           и list (в этом случае оставляет заголовки пустыми)."""
        # Проверяем тип переданного параметра и в зависимости от него по-разному инициализируем объект.
        if data==None:
            self.articles=[]
            self.dictionaries=[]
        if isinstance(data, BaseGetNewsPaper):
            self.articles=copy(data.articles)     
            self.dictionaries=copy(data.dictionaries)
        elif isinstance(data, list):
            self.articles=copy(data)
            self.dictionaries=[]
        # В любом случае создаем объект морфологии для создания частотных векторов.
        self.__morph=pymorphy2.MorphAnalyzer()
        self.__ttt=1

    # Построение вектора для статьи.
    def getArticleDictionary(self, text, needPos=None):
        """Строит частотные векторы для текста статьи, берет только значимые части речи."""
        words=[a[0] for a in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text)]
        reswords=[]
    
        for w in words:
            wordform=self.morph.parse(w)[0]
            try:
                if wordform.tag.POS in ['ADJF', 'NOUN', 'VERB', 'PRTF', 'GRND']:
                    if needPos!=None:
                        reswords.append(wordform.normal_form+'_'+wordform.tag.POS)
                    else:
                        reswords.append(wordform.normal_form)
            except:
                pass
            
        stat=Counter(reswords)
        # Берем только слова с частотой больше 1.
        stat={a: stat[a] for a in stat.keys() if stat[a]>1}
        return stat

    # Посчитаем вектора для всех статей.
    def calcArticleDictionaries(self, needPos=None):
        """Строит частотные вектора для всех статей в коллекции.
           !!! Не ясно что делать, когда пополняем. 
               По-хорошему надо хранить свойство, которое показывает 
               надо ли их считать для всех статей при добавлении или нет. !!!"""
        self.dictionaries=[]
        for a in self.articles:
            self.dictionaries.append(self.getArticleDictionary(a.text, needPos))
            
    # Абстрактный метод для загрузки новостей за заданный период.
    # Должен быть реализован в дочернем классе.
    @abstractmethod
    def getPeriod(self, start, finish):
        """Абстрактный метод для загрузки новостей за заданный период.
           Должен быть реализован в дочернем классе."""
        pass

    # Абстрактный метод для загрузки одной новости по ее адресу.
    # Должен быть реализован в дочернем классе.
    @abstractmethod
    def getArticle(self, url):
        """Абстрактный метод для загрузки одной новости по ее адресу.
        Должен быть реализован в дочернем классе."""
        pass

    # Сохраняем статьи в файл.
    def saveArticles(self, filename):
        """Сохраняет статью в файл с именем filename. 
           Статьи отделены друг от друга строкой "=====", заголовок от статьи строкой "-----". """
        newsfile = open(filename, "w")
        for art in self.articles:
            newsfile.write('\n=====\n'+art.title)
            newsfile.write('\n-----\n'+art.text)
        newsfile.close()

    # Читаем статьи из файла.
    def loadArticles(self, filename):
        """ Loads and replaces all articles from a file with a filename. """
        newsfile = open(filename, encoding="utf-8")
        text = newsfile.read()
        loaded = text.split('\n=====\n')[1:]
        self.articles=[]
        for i, a in enumerate(loaded):
            self.articles.append(BaseArticle())
            b, self.articles[i].text = a.split('\n-----\n')
            self.articles[i].title = b
        newsfile.close()
        
    # Показывает сколько статей загружено.
    def __len__(self):
        return len(self.articles)
    
    # Возвращает статью, если передано целое число или хранилище статей, если передан slice.
    def __getitem__(self, index):
        if type(index)==slice:
            return type(self)(self.articles[index])
        else:
            return self.articles[index]
        
    def __getattr__(self, prop):
        if len(prop)==1:
            if prop==prop.lower():
                return self.articles[ord(prop)-ord('a')].text
            else:
                return self.articles[ord(prop)-ord('A')].title
            
    def __lshift__(self, art):
        # Здесь надо что-то делать со словарями.
        if isinstance(art, BaseArticle):
            self.articles.append(art)
        elif isinstance(art, str):
            a1=BaseArticle()
            a1.text=art
            a1.title="No Title"
            self.articles.append(a1)
        else:
            raise NotImplementedError("Should be String or BaseArticle")
        return self
        
    def __iadd__(self, art):
        return self<<art
    
    def __add__(self, art):
        t=type(self)(self) # mtype=type(self), t=mtype(), t=self
        t+=art
        return t

    def __radd__(self, art):
        t=type(self)(self)
        t+=art
        return t
    
    @abstractmethod
    def __str__(self):
        pass
    
    def __call__(self):
        return len(self.articles)
    
    def __iter__(self):
        for art in self.articles:
            yield art
        return
    
    @property
    def morph(self):
        return self.__morph
        
class GetLenta(BaseGetNewsPaper):
    # Загрузка статьи по URL.
    def getLentaArticle(self, url):
        """ getLentaArticle gets the body of an article from Lenta.ru"""
        # Получает текст страницы.
        resp=requests.get(url)
        # Загружаем текст в объект типа BeautifulSoup.
        bs=BeautifulSoup(resp.text, "html5lib") 
        
        art=BaseArticle()
        # Получаем заголовок статьи.
        art.title=bs.h1.text.replace("\xa0", " ")
        # Получаем текст статьи.
        art.text=BeautifulSoup(" ".join([p.text for p in bs.find_all("p")]), "html5lib").get_text().replace("\xa0", " ")
        return art

    # Загрузка всех статей за один день.
    def getLentaDay(self, url):
        """ Gets all URLs for a given day and gets all texts. """
        try:
            # Грузим страницу со списком всех статей.
            day = requests.get(url) 
            # Получаем фрагменты с нужными нам адресами статей.
            h3s=BeautifulSoup(day.text, "html5lib").find_all("h3")
            # Получаем все адреса на статьи за день.
            links=["http://lenta.ru"+l.find_all("a")[0]["href"] for l in h3s]
            # Загружаем статьи.
            for l in links:
                art=self.getLentaArticle(l)
                self.articles.append(art)
        except:
            pass

    # Загрузка всех статей за несколько дней.
    def getLentaPeriod(self, start, finish):
        curdate=start
        while curdate<=finish:
            print(curdate.strftime('%Y/%m/%d')) # Just in case.
            # Список статей грузится с вот такого адреса.
            self.getLentaDay('https://lenta.ru/news/'+curdate.strftime('%Y/%m/%d'))
            curdate+=datetime.timedelta(days=1)

    def getPeriod(self, start, finish):
        self.getLentaPeriod(start, finish)
        
    def __str__(self):
        return "<Lenta.ru scrapper: "+str(len(self.articles))+" articles loaded>"
    
    def getArticle(self, url):
        if "lenta.ru" in url.lower():
            self.lower().getLentaArticle(url)
        else:
            raise NotImplementedError("I can download from Lenta.ru site only.")
            

class GetNPlus1(BaseGetNewsPaper):
    def getArticleTextNPlus1(self, adr):
        r = requests.get(adr)
        #print(r.text)
        art = NPlus1Article()
        tables = re.split("</div>",re.split('="tables"', r.text)[1])[0]
        t1 = re.split("</time>", re.split("<time", tables)[1])[0]
        art.time = re.split("</span>", re.split("<span>", t1)[1])[0]
        art.date = re.split("</span>", re.split("<span>", t1)[2])[0]
        art.rubr = re.findall("<a data-rubric.+?>(.+?)</a>", r.text)[0]
        art.diff = re.split("</span>", re.split('"difficult-value">', tables)[1])[0]
        art.title = re.findall("<h1>(.+?)</h1>", r.text)[0]
        art.author = re.split('" />',re.split('<meta name="author" content="', r.text)[1])[0]
        art.text = re.split("</div>", re.split("</figure>", re.split('</article>',re.split('<article', r.text)[1])[0])[1])[1]    

        beaux_text = BeautifulSoup(art.text, "html5lib")
        art.text = delcom.sub("", beaux_text.get_text() )
        art.text = art.text.replace('\xa0', ' ')
        return art

    def getNPlus1Day(self, adr):
        r = requests.get(adr)
        titles = BeautifulSoup(r.text, "html5lib")("article")
        addrs = ["https://nplus1.ru/"+a("a")[0]["href"] for a in titles]
        for adr in addrs:
            aa=self.getArticleTextNPlus1(adr)
            self.articles.append(aa)
        
    # Загрузка всех статей за несколько дней.
    def getNPlus1Period(self, start, finish):
        curdate=start
        while curdate<=finish:
            print(curdate.strftime('%Y/%m/%d')) # Just in case.
            # Список статей грузится с вот такого адреса.
            self.getNPlus1Day('https://nplus1.ru/news/'+curdate.strftime('%Y/%m/%d'))
            curdate+=datetime.timedelta(days=1)

    def getPeriod(self, start, finish):
        self.getNPlus1Period(start, finish)
        
    def __str__(self):
        return "<NPlus1.ru scrapper: "+str(len(self.articles))+" articles loaded>"

    def getArticle(self, url):
        if "nplus1.ru" in url.lower():
            return self.getArticleTextNPlus1(url)
        else:
            raise NotImplementedError("I can download from Lenta.ru site only.")
    

In [26]:
a1=BaseArticle()
a2=BaseArticle()
a1.title="123"
a2.title="321"
print(a1, a2)
a3=BaseArticle(a1, a2)
print(a3)

<title: 123
text: ... > <title: 321
text: ... >
<title: 
text: ... >


In [27]:
n1=GetNPlus1()
n1.loadArticles("data/nplus1_test.txt")
# Проверяем как работает __len__()
#len(n1)

# Проверяем как работают срезы.
#len(n1[1:5])
#type(n1[1:5])
#n1[1].title

# Инициализирующий конструктор.
#n2=GetNPlus1(n1)
#n2.articles[0].title, n1.articles[0].title

# Выдача свойств, которых нет у объекта - спорная практическая значимость и очевидность кода.
#n1.A, n1.b

# Работа с @property
#print(n1.morph)
#print(n1.__morph)

# Операторы сдвига и сложения для разных типов.
#n1<<n1[0]
#n1+=n1[0]
#n1<<1
#print("", n1.A, "\n", n1[-1].title)

# "Левое" сложение против "правого".
#n2=n1+n1[0]
#n2=n1[0]+n1
#print("", n2.A, "\n", n2[-1].title)

# Преобразование к строке.
#print(n1)

# "Вызов" функции как объекта.
#n1()

# Тестируем __str__()
#print(n1.getArticle("https://nplus1.ru/news/2019/02/20/deep-sqeak"))
# Тестируем __repr__()
#n1[0]

# Тестируем коллекцию статей как итерируемый объект.
for art in n1[2:10]:
    print(art, "\n")

<title: Беспилотные автомобили Waymo оказались самыми самостоятельными
text: Беспилотные автомобили компании Waymo оказались самыми самостоятельными в 2017 году — в среднем водителям-испытателям приходилось перехватывать управление один раз в почти девять тысяч километров, в т... > 

<title: Неупорядоченная структура шелка сделала его блестящим и холодным
text: Физики обнаружили, что блеск шелка возникает из-за наличия в нитях неупорядоченных полостей нанометровой толщины,на которых происходит интерференция света. Та же причина объясняет и теплообмен в нитях... > 

<title: Беспилотный автомобиль испытают британскими дорогами
text: Автомобильные компании Nissan, Renault и Mitsubishi совместно с Университетом Крэнфилда и управляющей компанией Highways England объявили о намерении провести испытания беспилотного автомобиля левосто... > 

<title: CRISPR заставит биться сердца больных мышечной дистрофией Дюшенна
text: Исследователи отредактировали клетки сердечной мышцы с мутациями, приводя

Большим удивлением для меня было узнать, что свойства с двумя подчеркиваниями перестали быть видимыми в новых версиях Python. Пользуйтесь свойствами!

In [28]:
class Test1:
    def __init__(self):
        self.__x=0
        self.__y=0
    
    def printX(self):
        print(self.__x)
        
tst=Test1()
tst.printX()
print("===")
print(tst.__x)


0
===


AttributeError: 'Test1' object has no attribute '__x'

Но при помощи [библиотек](https://habr.com/ru/post/443192/) можно добиться еще большего.